# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_deterministic.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data="collected_data/rl_stochpid2.txt")
    samples.setting("double")
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::2222] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset = get_dataset([i for i in range(2000)])

start
[ 0.00000000e+00  7.95731469e+08 -1.30891077e-02  1.00000047e-02
 -1.13000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.18111235e-01  7.86276171e-02  2.40256998e-01]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.14710892e-01  4.26000047e-02
  2.94999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.14710892e-01  4.26000047e-02
  2.94999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 3 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.13891077e-02  2.58000047e-02
 -1.58000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21545490e-01  4.84695201e-02  4.95970982e-01]
Read chunk # 4 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.13891077e-02  2.58000047e-02
 -1.58000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21545490

[ 0.00000000e+00  7.95731469e+08 -4.87891077e-02  1.90000047e-02
 -2.46000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  1.20983127e-01  6.00000000e-01]
Read chunk # 75 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.12089108e-01 -5.99999953e-02
 -4.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.16286537e-01 -2.80152941e-01  4.76846770e-01]
Read chunk # 76 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.12089108e-01 -5.99999953e-02
 -4.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.16286537e-01 -2.80152941e-01  4.76846770e-01]
Read chunk # 77 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.10889108e-01  2.00004692e-04
  2.79999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.42296700e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 78 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.10889108e-01  2.00004692e-04
  2.79999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.42296700e-

[ 0.00000000e+00  7.95731469e+08  1.99610892e-01 -7.79999531e-03
 -1.56000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 151 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.48910892e-01 -2.61999953e-02
 -9.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.19613549e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 152 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.48910892e-01 -2.61999953e-02
 -9.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.19613549e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 153 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.92789108e-01  2.40000047e-02
 -2.32000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -4.58171273e-01  6.00000000e-01]
Read chunk # 154 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.92789108e-01  2.40000047e-02
 -2.32000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.000000

start
[ 0.00000000e+00  7.95731469e+08  4.14710892e-01 -1.99999953e-02
 -2.27000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.62293447e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 191 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.23610892e-01  1.28000047e-02
  7.69998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.99715337e-01  2.22184822e-01 -6.00000000e-01]
Read chunk # 192 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.23610892e-01  1.28000047e-02
  7.69998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.99715337e-01  2.22184822e-01 -6.00000000e-01]
Read chunk # 193 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.42889108e-01  4.82000047e-02
 -2.11000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 194 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.42889108e-01  4.82000047e-02
 -2.11000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.

[ 0.00000000e+00  7.95731469e+08  3.76310892e-01  4.40000047e-02
  8.69998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.40916293e-02  6.00000000e-01 -6.00000000e-01]
Read chunk # 265 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  7.75108923e-02 -5.89999953e-02
  1.05999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.21981698e-01  3.04380441e-01 -6.00000000e-01]
Read chunk # 266 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  7.75108923e-02 -5.89999953e-02
  1.05999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.21981698e-01  3.04380441e-01 -6.00000000e-01]
Read chunk # 267 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.47789108e-01 -5.55999953e-02
  5.49998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.92820940e-01 -3.46694881e-01  1.80867451e-01]
Read chunk # 268 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.47789108e-01 -5.55999953e-02
  5.49998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.928209

...

start
[ 0.00000000e+00  7.95731469e+08 -4.03689108e-01  4.50000047e-02
 -2.30000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1711 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.03689108e-01  4.50000047e-02
 -2.30000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1712 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.20789108e-01  5.52000047e-02
 -2.96000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -4.10363558e-02  6.00000000e-01]
Read chunk # 1713 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.20789108e-01  5.52000047e-02
 -2.96000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -4.10363558e-02  6.00000000e-01]
Read chunk # 1714 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.93889108e-01 -9.19999531e-03
 -1.52000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


start
[ 0.00000000e+00  7.95731469e+08 -1.25589108e-01 -4.79999953e-02
 -4.70001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.53771117e-02 -5.92966477e-01  6.00000000e-01]
Read chunk # 1782 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.24610892e-01  1.60000469e-03
  8.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.11119783e-04  4.27170387e-01 -3.45557112e-01]
Read chunk # 1783 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.24610892e-01  1.60000469e-03
  8.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.11119783e-04  4.27170387e-01 -3.45557112e-01]
Read chunk # 1784 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.39310892e-01 -4.71999953e-02
  2.62999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -1.64116219e-03 -6.00000000e-01]
Read chunk # 1785 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.39310892e-01 -4.71999953e-02
  2.62999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


Read chunk # 1853 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.06108923e-02 -3.81999953e-02
  1.17999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -1.92536450e-01  3.03176353e-01]
Read chunk # 1854 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -6.82891077e-02 -1.05999953e-02
 -2.54000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -1.97565638e-02  6.00000000e-01]
Read chunk # 1855 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -6.82891077e-02 -1.05999953e-02
 -2.54000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -1.97565638e-02  6.00000000e-01]
Read chunk # 1856 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.32989108e-01 -4.37999953e-02
  2.01999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  5.59395707e-01]
Read chunk # 1857 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.32989108e-01 -4.37999953e-02
  2.01999866e-02  0.00000000e+00 -5

[ 0.00000000e+00  7.95731469e+08 -1.16289108e-01  3.04000047e-02
  1.32999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.22124397e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 1927 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.16289108e-01  3.04000047e-02
  1.32999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.22124397e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 1928 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.88891077e-02 -4.85999953e-02
 -2.26000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -7.82786801e-02  1.69427052e-01]
Read chunk # 1929 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.88891077e-02 -4.85999953e-02
 -2.26000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -7.82786801e-02  1.69427052e-01]
Read chunk # 1930 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.99210892e-01  1.02000047e-02
  2.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.52

[ 0.00000000e+00  7.95731469e+08 -3.58989108e-01 -5.17999953e-02
  3.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.19684716e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 2000 out of 4999
torch.Size([2220000, 6])


In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -248.49933,
 'std': 154.67868,
 'min': -752.22614,
 'max': 0.0,
 'histogram': (array([ 10,   9,   6,   9,  17,  16,  29,  34,  32,  39,  38,  62,  57,
          85,  94, 116, 155, 177,  14,   1]),
  array([-752.22614 , -714.6148  , -677.00354 , -639.3922  , -601.7809  ,
         -564.1696  , -526.5583  , -488.947   , -451.3357  , -413.72437 ,
         -376.11307 , -338.50177 , -300.89044 , -263.27914 , -225.66785 ,
         -188.05653 , -150.44522 , -112.83392 ,  -75.22261 ,  -37.611305,
            0.      ], dtype=float32))}

In [5]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [6]:
from d3rlpy.algos import CQL

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [7]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.06862036204800842


In [8]:
%load_ext tensorboard
%tensorboard --logdir runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/dasc/anaconda3/envs/jbreeden3.10/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/main.py", line 40, in <module>
    from tensorboard import default
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/default.py", line 38, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard import plugin_util
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 21, in <module>
    from tensorboard._vendor import bleach
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorb

In [9]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=40, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-20 15:33.35 [debug    ] RoundIterator is selected.
2022-04-20 15:33.35 [info     ] Directory is created at d3rlpy_logs/CQL_20220420153335
2022-04-20 15:33.35 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-20 15:33.35 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-20 15:33.35 [warning  ] Skip building models since they're already built.
2022-04-20 15:33.35 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220420153335/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/40:   0%|          | 0/6931 [00:00<?, ?it/s]

2022-04-20 15:43.19 [info     ] CQL_20220420153335: epoch=1 step=6931 epoch=1 metrics={'time_sample_batch': 0.0003195364501737375, 'time_algorithm_update': 0.08248434316508269, 'temp_loss': 2.768967193367677, 'temp': 0.747011374728949, 'alpha_loss': -3.7443262735867933, 'alpha': 1.2409134573827074, 'critic_loss': 19.95090428755373, 'actor_loss': 24.710188812992985, 'time_step': 0.08313859886738528, 'td_error': 33.174810272332685, 'init_value': -63.10578155517578, 'ave_value': -54.6618558456178} step=6931
2022-04-20 15:43.19 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220420153335/model_6931.pt


Epoch 2/40:   0%|          | 0/6931 [00:00<?, ?it/s]

2022-04-20 15:53.46 [info     ] CQL_20220420153335: epoch=2 step=13862 epoch=2 metrics={'time_sample_batch': 0.00032477519192610995, 'time_algorithm_update': 0.0886658003381485, 'temp_loss': 0.9284023452228221, 'temp': 0.4178826983761708, 'alpha_loss': 0.5071393584290255, 'alpha': 1.1343545683739833, 'critic_loss': 108.15097504456035, 'actor_loss': 88.52169504579811, 'time_step': 0.0893653748338485, 'td_error': 70.28582000488592, 'init_value': -123.6652603149414, 'ave_value': -108.84911071964142} step=13862
2022-04-20 15:53.46 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220420153335/model_13862.pt


...

Epoch 39/40:   0%|          | 0/6931 [00:00<?, ?it/s]

2022-04-20 22:06.38 [info     ] CQL_20220420153335: epoch=39 step=270309 epoch=39 metrics={'time_sample_batch': 0.0003423051320875281, 'time_algorithm_update': 0.09143564601667584, 'temp_loss': 0.00021143193109988384, 'temp': 0.13938470317453658, 'alpha_loss': 0.0018873487403012722, 'alpha': 0.22648190581519922, 'critic_loss': 35.705481725614696, 'actor_loss': 17.279242882281736, 'time_step': 0.09216279901651193, 'td_error': 16.95232798074316, 'init_value': -16.499317169189453, 'ave_value': -13.00099126757284} step=270309
2022-04-20 22:06.39 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220420153335/model_270309.pt


Epoch 40/40:   0%|          | 0/6931 [00:00<?, ?it/s]

2022-04-20 22:17.39 [info     ] CQL_20220420153335: epoch=40 step=277240 epoch=40 metrics={'time_sample_batch': 0.00033363634125182953, 'time_algorithm_update': 0.09320847132628725, 'temp_loss': 0.0001576871940318687, 'temp': 0.13916002034650807, 'alpha_loss': -0.0005196051136630011, 'alpha': 0.22511139057996968, 'critic_loss': 35.590238842995475, 'actor_loss': 17.075815960790685, 'time_step': 0.09389307945773227, 'td_error': 16.91298724805621, 'init_value': -16.443775177001953, 'ave_value': -13.03374250195694} step=277240
2022-04-20 22:17.39 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220420153335/model_277240.pt


[(1,
  {'time_sample_batch': 0.0003195364501737375,
   'time_algorithm_update': 0.08248434316508269,
   'temp_loss': 2.768967193367677,
   'temp': 0.747011374728949,
   'alpha_loss': -3.7443262735867933,
   'alpha': 1.2409134573827074,
   'critic_loss': 19.95090428755373,
   'actor_loss': 24.710188812992985,
   'time_step': 0.08313859886738528,
   'td_error': 33.174810272332685,
   'init_value': -63.10578155517578,
   'ave_value': -54.6618558456178}),
 (2,
  {'time_sample_batch': 0.00032477519192610995,
   'time_algorithm_update': 0.0886658003381485,
   'temp_loss': 0.9284023452228221,
   'temp': 0.4178826983761708,
   'alpha_loss': 0.5071393584290255,
   'alpha': 1.1343545683739833,
   'critic_loss': 108.15097504456035,
   'actor_loss': 88.52169504579811,
   'time_step': 0.0893653748338485,
   'td_error': 70.28582000488592,
   'init_value': -123.6652603149414,
   'ave_value': -108.84911071964142}),
 (3,
  {'time_sample_batch': 0.00032805914900172514,
   'time_algorithm_update': 0.0977

In [10]:
model.save_model('cqlStochpidDouble2000_Ep40_CPUOnly.pt')
model.save_policy('cqlStochpidDobule2000_Ep40_CPUOnly.pt')

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [11]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i+2000 for i in range(100)]) #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [12]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i*2 for i in range(100)], path="collected_data/rl_stochastic.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [13]:
# from d3rlpy.torch_utility import to_cpu
# to_cpu(model)
# model.save_policy("cqlStochpid2000Ep40CPU.pt")
# model.save_model("cqlStochpid2000Ep40modelCPU.pt")

In [14]:
# for key in dir(model):
#     module = getattr(model, key)
#     if isinstance(module, (torch.nn.Module, torch.nn.Parameter)):
#         print(yes)
#         print(key)
# dir(model)
# type(model)
# model.cpu()
# from d3rlpy.algos.torch.base import TorchImplBase
# new_model = TorchImplBase()
# from d3rlpy.torch_utility import _get_attributes
# model._device = "cpu:0"
# print(model._device)


# def my_get_state_dict(impl: Any) -> Dict[str, Any]:
#     rets = {}
#     for key in _get_attributes(impl):
#         obj = getattr(impl, key)
#         if isinstance(obj, (torch.nn.Module, torch.optim.Optimizer)):
#             if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#                 obj.cpu()
#             rets[key] = obj.state_dict()
#     return rets

# torch.save(my_get_state_dict(model), "my_test_model.pt")

# for key in dir(model):
#     obj = getattr(model, key)
#     if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#         obj.cpu()
#         print("convert to cpu")
# model.save_policy("cqlStochpid2000Ep40modelCPU.pt")

# import trace
# tracer = trace.Trace()
# tracer.run('model.save_policy("cqlStochpid2000Ep40modelCPU.pt")')